## PREDICATE: Predictor of Antiviral Targets

Predict metabolic host-based enzymatic targets of a one or more viral genome sequences of a single virus.

### Steps:
- Introduce amino acid mutations to reference protein sequence
- Knock-out experiments
- Host-derived enforcement 
- In case of multiple input sequences: Take for each target mean of predicted inhibitory effect over input sequences
- Visualizations

In [ ]:
import os
import cobra
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import single_reaction_deletion
from cobra.flux_analysis import flux_variability_analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from Bio import SeqIO
import matplotlib as mpl
from matplotlib.ticker import MaxNLocator
import matplotlib.patches as mpatches
import seaborn as sns

#### Define constant variables:

In [ ]:
# please provide paths to your input files in the empty strings

# name for output files
variant_name = ' '

# one or more mutated nucleotide sequences
fasta_sequences_file = ' '

# contain mutation info as extracted from GISAID 
# (optional; in case reference nucleotide sequence is used the variable is defined as empty string)
csv_table = ' '

# reference (not mutated) sequence
ref_rec = list(SeqIO.parse(' ', "fasta"))

# metabolic network in SBML format
model = cobra.io.read_sbml_model(' ')

# reference protein sequence 
records = list(SeqIO.parse(' ', "fasta"))

# ids of known structural proteins
structural = ['lcl|NC_045512.2_prot_YP_009724390.1_3', 'lcl|NC_045512.2_prot_YP_009724392.1_5', 
              'lcl|NC_045512.2_prot_YP_009724393.1_6', 'lcl|NC_045512.2_prot_YP_009724397.2_11']

# amino acids
amino_acids = ['A', 'R', 'N', 'D', 'C', 'E', 'Q', 'G', 'H', 'I', 'L', 'K', 'M', 
               'F', 'P', 'S', 'T', 'W', 'Y', 'V']

# modelcular weights of amino acids, taken from CheBI
molecular_weight = [89.1, 174.2, 132.1, 133.1, 121.2, 147.1, 146.2, 75.1, 155.2, 131.2, 131.2, 
                    146.2, 149.2, 165.2, 115.1, 105.1, 119.1, 204.2, 181.2, 117.1]

# Copy numbers of structural and non-structural proteins
Cs = 120
Ce = 20
Cn = 456
Cm = 1000
Cnp = 1

# genome copy number
Cg = 1 

# 4 ATP molecules are needed for the polymerization of amino acids
kATP = 4 

# 1 PPI molecule is needed to bond two nulceotides
kppi = 1 

#### Create directory to store results:

In [ ]:
path = "test_Results/"

os.mkdir(path)

#### Store all input sequences:

In [ ]:
all_sequences = {}

# read fasta file
fasta_sequences = SeqIO.parse(open(fasta_sequences_file),'fasta')

for fasta in fasta_sequences:
    
    # store sequence id and sequence
    name, sequence = fasta.id, str(fasta.seq)
    
    # add info to dictionary
    all_sequences[name] = sequence
    
print(all_sequences)

#### Analyse nucleotide counts and type of mutations + visualize: 

In [ ]:
# if multiple input sequences 
if csv_table != " ":
    
    # extract mutations info
    data_df = pd.read_table(csv_table,sep=';')
    print("data_df: ", data_df)
    
    # create df to store nucleotides count
    nucleotide_counts_df = pd.DataFrame(columns=['GISAID Accession ID', 'A','T','G','C'])

    for name, seq in all_sequences.items():

        new_row = {'GISAID Accession ID':data_df.loc[data_df['strain']==name]['gisaid_epi_isl'].values[0], 'A':seq.count("A"), 'T':seq.count("T"), 'G':seq.count("G"),'C':seq.count("C")}
        nucleotide_counts_df = nucleotide_counts_df.append(new_row, ignore_index=True)

    nucleotide_counts_df = nucleotide_counts_df.sort_values(by=["A","T","G","C"])

    ################################################################################################
    # Additional (optional) Calculations: calculate log2FC between mutated and not-mutated sequence
    ################################################################################################
    ref_seq = ref_rec[0].seq

    ref_seq_ncl_counts = [ref_seq.count("A"),ref_seq.count("T"),ref_seq.count("G"),ref_seq.count("C")]
    print("ref_seq_ncl_counts:", ref_seq_ncl_counts)

    log2FC_A,log2FC_T,log2FC_G,log2FC_C = [],[],[],[]
    for index,row in nucleotide_counts_df.iterrows():

        log2FC_A.append(np.log2(row['A']/ref_seq_ncl_counts[0]))
        log2FC_T.append(np.log2(row['T']/ref_seq_ncl_counts[1]))
        log2FC_G.append(np.log2(row['G']/ref_seq_ncl_counts[2]))
        log2FC_C.append(np.log2(row['C']/ref_seq_ncl_counts[3]))

    nucleotide_counts_df['log2FC_A'] = log2FC_A
    nucleotide_counts_df['log2FC_T'] = log2FC_T
    nucleotide_counts_df['log2FC_G'] = log2FC_G
    nucleotide_counts_df['log2FC_C'] = log2FC_C
    
    print("nucleotide_counts_df: ", nucleotide_counts_df)
    
        
    ##########################
    # Plot nucleotide counts
    ##########################
    mpl.rcParams['axes.spines.right'] = False

    plt.rcParams['font.size'] = '16'

    nucleotide_counts_df[['GISAID Accession ID','log2FC_A','log2FC_T','log2FC_G','log2FC_C']].plot(x='GISAID Accession ID',
            kind='bar',
            stacked=False,
            figsize=(19,10),
            color=["#d7191c", "#fdae61", "#abdda4", "#2b83ba"],
            width=0.7)

    plt.axhline(0.0, color='k', linewidth=1.2)
    plt.legend(loc="best")
    plt.xticks(rotation=89)
    plt.ylabel("Counts")

    plt.tight_layout()

    plt.savefig(path+"all_sequences_nucleotides_counts_with_log2FC.pdf")
    plt.savefig(path+"all_sequences_nucleotides_counts_with_log2FC.png")

    plt.show()
    
    ########################
    # Analyse Mutations
    #######################
    mutation_data = data_df['AA Mutations']
    print("mutation_data: ", mutation_data)
    
    # Important check: do all mutations involve letters related to amino acids or involve also "unknown" letters
    # allowed: all amino acid letters + del, stop, ins, dupl words + numbers
    allowed_chars = amino_acids+['s','t','o','p','d','e','l','i','n','s','u']+['0','1','2','3','4','5','6','7','8','9']

    c = 0
    for d in mutation_data:

        for m in d.split(","):

            for char in m.strip().split(" ")[1]:

                if char not in allowed_chars:

                    # print character and index of occured sequence
                    print('weird char detected: ', char, c)
        c+=1
    
    
    mutations_df = pd.DataFrame(columns=['Virus Name', 'GISAID Accession ID', 'Mutations'])

    mutations_df['Virus Name'] = data_df['strain']
    mutations_df['GISAID Accession ID'] = data_df['gisaid_epi_isl']
    mutations_df['Mutations'] = data_df['AA Mutations']
    mutations_df.to_csv(path+"mutations_info_"+variant_name+"_mean_VBOF.csv")
    print('mutation_df: ', mutations_df)


    # store types of mutations observed in the 20 downloaded sequences
    mut_types = []

    for data in mutation_data:

        for mut in data.split(","):

            if mut.strip() not in mut_types:

                mut_types.append(mut.strip())

    print('\nmut_types: ', mut_types)

    # count how oft a mutation appears
    mut_frequencies = {}
    for mut in mut_types:

        count = 0

        for data in mutation_data:

            if mut in data:

                count +=1

        mut_frequencies[mut] = count

    mut_frequencies = dict(sorted(mut_frequencies.items(), key=lambda item: item[1]))
    print('\nmut_frequencies: ', mut_frequencies)
    
        
    ##############################
    # Plot Mutations Information
    ##############################
    plt.figure(figsize=(19,8)).gca().yaxis.set_major_locator(MaxNLocator(integer=True))
    plt.rcParams['font.size'] = '19'

    names = list(mut_frequencies.keys())
    values = list(mut_frequencies.values())

    barplot = plt.bar(range(len(mut_frequencies)), values, tick_label=names, color="#d7191c")

    # highlight different mutations
    for key,value in mut_frequencies.items():

        if 'del' in key:
            barplot[list(mut_frequencies).index(key)].set_color('#3288bd')

        if 'stop' in key:
            barplot[list(mut_frequencies).index(key)].set_color('#99d594')

        if 'ins' in key:
            barplot[list(mut_frequencies).index(key)].set_color('#fc8d59')

        if 'dupl' in key:
            barplot[list(mut_frequencies).index(key)].set_color('#984ea3')


    plt.xlim([-0.9,len(mut_frequencies)])
    plt.ylim([0,21])

    plt.xticks(rotation=89)

    plt.ylabel("Counts")
    plt.xlabel("Mutation Type")

    blue_patch = mpatches.Patch(color='#3288bd', label='Deletions')
    green_patch = mpatches.Patch(color='#99d594', label='Involves stop codon')
    plt.legend(handles=[blue_patch, green_patch])

    plt.tight_layout()

    plt.savefig(path+variant_name+"_sequences_mutation_counts.pdf")
    plt.savefig(path+variant_name+"_sequences_mutation_counts.png")

    plt.show()
    
    ###########################################
    # Plot Mutations Information per category
    ###########################################
    # store mutations per protein
    spike_prot_muts, n_prot_muts, m_prot_muts, e_prot_muts,nsp_prot_muts, ns_prot_muts = {},{},{},{},{},{}

    for key, value in mut_frequencies.items():
        if "Spike" in key:
            spike_prot_muts[key.split("Spike")[1].strip()] = value

        elif "N " in key:
            n_prot_muts[key.split("N")[1].strip()] = value

        elif "M " in key:
            m_prot_muts[key.split("M")[1].strip()] = value

        elif "E " in key:
            e_prot_muts[key.split("E")[1].strip()] = value

        elif "NSP" in key:
            nsp_prot_muts[key] = value

        else:
            ns_prot_muts[key] = value

    print("spike_prot_muts", spike_prot_muts)
    print("n_prot_muts", n_prot_muts)
    print("m_prot_muts", m_prot_muts)
    print("e_prot_muts", e_prot_muts)
    print("nsp_prot_muts", nsp_prot_muts)
    print("ns_prot_muts", ns_prot_muts)
    print("\n Total length", len(mut_frequencies))
    print("\n Total length", len(spike_prot_muts)+len(n_prot_muts)+len(m_prot_muts)+ len(e_prot_muts)+ len(nsp_prot_muts)+ len(ns_prot_muts))
    
    
    plt.rcParams['font.size'] = 18
    plt.rcParams['xtick.labelsize'] = 18

    colors=["#d53e4f", "#fc8d59", "#fee08b", "#e6f598", "#99d594", "#3288bd"]

    fig, axes = plt.subplots(2, 3, figsize=(30, 20), sharey=True)

    plt.subplots_adjust(left=0.125,
                        bottom=0.1, 
                        right=0.9, 
                        top=0.9, 
                        wspace=0.2, 
                        hspace=0.35)

    sns.barplot(ax=axes[0,0], x=list(spike_prot_muts.keys()), y=list(spike_prot_muts.values()),color=colors[0])
    axes[0,0].set_title("Spike Protein", fontweight="bold")
    axes[0,0].set_xticklabels(list(spike_prot_muts.keys()), rotation=90)

    sns.barplot(ax=axes[0,1], x=list(n_prot_muts.keys()), y=list(n_prot_muts.values()), color=colors[1])
    axes[0,1].set_title("N Protein", fontweight="bold")
    axes[0,1].set_xticklabels(list(n_prot_muts.keys()), rotation=90)

    sns.barplot(ax=axes[0,2], x=list(m_prot_muts.keys()), y=list(m_prot_muts.values()), color=colors[2])
    axes[0,2].set_title("Membrane Protein", fontweight="bold")
    axes[0,2].set_xticklabels(list(m_prot_muts.keys()), rotation=90)

    if len(e_prot_muts) != 0:
        # ! omit if no mutation on the E protein found (e.g., in gamma SARS-CoV-2 variant)
        sns.barplot(ax=axes[1,0], x=list(e_prot_muts.keys()), y=list(e_prot_muts.values()), color=colors[3])
        axes[1,0].set_title("Envelope Protein", fontweight="bold")
        axes[1,0].set_xticklabels(list(e_prot_muts.keys()), rotation=90)

    sns.barplot(ax=axes[1,1], x=list(nsp_prot_muts.keys()), y=list(nsp_prot_muts.values()), color=colors[4])
    axes[1,1].set_title("NSP Proteins",fontweight="bold")
    axes[1,1].set_xticklabels(list(nsp_prot_muts.keys()), rotation=90)

    sns.barplot(ax=axes[1,2], x=list(ns_prot_muts.keys()), y=list(ns_prot_muts.values()), color=colors[5])
    axes[1,2].set_title("NS Ptoteins", fontweight="bold")
    axes[1,2].set_xticklabels(list(ns_prot_muts.keys()), rotation=90)

    plt.tight_layout()

    plt.savefig(path+variant_name+"_sequences_mutation_counts_alltogether.pdf")
    plt.savefig(path+variant_name+"_sequences_mutation_counts_alltogether.png")

    plt.show()

# if only one input sequence, simply count nucleotides
else:
    
    nuc_seq = list(all_sequences.values())[0]
    print("A: ", nuc_seq.count("A"))
    print("T: ", nuc_seq.count("T"))
    print("G: ", nuc_seq.count("G"))
    print("C: ", nuc_seq.count("C"))
    print("Total length: ",len(seq))  

In [ ]:
def host_derived_enforcement(model, vbof_name, bof_name, growth_rate_vbof, growth_rate_bof):
    
    """ 
    Host-derived Enforcement
    
    Inputs:
        - model: SBML model 
        - vbof_name: string of vbof ID in given model
        - bof_name: string of bof ID in given model
        - growth_rate_vbof: virus growth rate 
        - growth_rate_bof: host maintenance rate
        
    Outputs:
        - data: list of lists with all predicted targets and the respective % reamining BOF
        
    """
    
    model.objective=bof_name
    print('BOF optimized:', model.optimize().objective_value)
    fva_mac = flux_variability_analysis(model,fraction_of_optimum=0.98)
    fva_mac =fva_mac.reset_index()
    fva_mac = fva_mac.rename(columns={'index': 'Reaction_ID', 'minimum': 'H_(F-)', 'maximum': 'H_(F+)'})

    model.objective=vbof_name
    print('VBOF optimized:', model.optimize().objective_value)
    fva_vbof = flux_variability_analysis(model,fraction_of_optimum=0.75)
    fva_vbof =fva_vbof.reset_index()
    fva_vbof = fva_vbof.rename(columns={'index': 'Reaction_ID', 'minimum': 'V_(F-)', 'maximum': 'V_(F+)'})

    fva = fva_mac.merge(fva_vbof, left_on='Reaction_ID', right_on='Reaction_ID')
    fba_vbof_fluxes = model.optimize().fluxes

    hde = {}
    zero_fba_vbof_flux = []

    for i in range(len(fva)):

        if fva.loc[i]['H_(F+)'] == fva.loc[i]['V_(F+)'] and fva.loc[i]['H_(F-)'] == fva.loc[i]['V_(F-)']:
            continue

        elif fva.loc[i]['H_(F+)'] > fva.loc[i]['V_(F+)'] and fva.loc[i]['H_(F-)'] >= fva.loc[i]['V_(F-)']:
            new_lower_bound = fva.loc[i]['H_(F+)'] - ((fva.loc[i]['H_(F+)']-fva.loc[i]['V_(F+)'])/2)
            new_upper_bound = fva.loc[i]['H_(F+)']

        elif fva.loc[i]['H_(F-)'] < fva.loc[i]['V_(F-)'] and fva.loc[i]['H_(F+)'] <= fva.loc[i]['V_(F+)']:
            new_lower_bound = fva.loc[i]['H_(F-)']
            new_upper_bound = fva.loc[i]['H_(F-)'] - ((fva.loc[i]['H_(F-)']-fva.loc[i]['V_(F-)'])/2)

        elif fva.loc[i]['H_(F-)'] > fva.loc[i]['V_(F-)'] and fva.loc[i]['H_(F+)'] < fva.loc[i]['V_(F+)']:
            new_lower_bound = fva.loc[i]['H_(F-)']
            new_upper_bound = fva.loc[i]['H_(F+)']

        elif fva.loc[i]['H_(F-)'] >= fva.loc[i]['V_(F-)'] and fva.loc[i]['H_(F+)'] <= fva.loc[i]['V_(F+)']:
            new_upper_bound = fva.loc[i]['H_(F+)'] - ((fva.loc[i]['H_(F+)']-fva.loc[i]['V_(F+)'])/2)
            new_lower_bound = fva.loc[i]['H_(F-)'] - ((fva.loc[i]['H_(F-)']-fva.loc[i]['V_(F-)'])/2)   

        else:
            print('else: ', fva.loc[i]['Reaction_ID'])


        with model: 
            # store old (initial) reaction bounds
            old_lb = model.reactions.get_by_id(fva.loc[i]['Reaction_ID']).lower_bound
            old_ub = model.reactions.get_by_id(fva.loc[i]['Reaction_ID']).upper_bound

            direction_before = model.reactions.get_by_id(fva.loc[i]['Reaction_ID']).reaction.split(" ")

            # ensure that lower bound will always be < than upper bound
            if new_lower_bound > new_upper_bound:
                model.reactions.get_by_id(fva.loc[i]['Reaction_ID']).lower_bound = round(new_upper_bound,6)
                model.reactions.get_by_id(fva.loc[i]['Reaction_ID']).upper_bound = round(new_lower_bound,6)
            else:
                model.reactions.get_by_id(fva.loc[i]['Reaction_ID']).lower_bound = round(new_lower_bound,6)
                model.reactions.get_by_id(fva.loc[i]['Reaction_ID']).upper_bound = round(new_upper_bound,6)

            direction_after = model.reactions.get_by_id(fva.loc[i]['Reaction_ID']).reaction.split(" ")

            # optimize BOF 
            model.objective = bof_name
            host = model.optimize().objective_value
            # optimize VBOF 
            model.objective = vbof_name
            virus = model.optimize().objective_value
                    
            # report reactions that reduce the virus growth rate to below 50% of its initial growth rate
            if virus < 0.5*growth_rate_vbof:
                
                print(direction_before,direction_after)


                if fba_vbof_fluxes[fva.loc[i]['Reaction_ID']] == 0.0:

                    # ignore those, since they don't influence the virus's growth
                    print('Zero FBA flux when VBOF optimized: ', fva.loc[i]['Reaction_ID'])
                    zero_fba_vbof_flux.append(fva.loc[i]['Reaction_ID'])

                else:

                    if  new_lower_bound >= 0 and new_upper_bound > 0:
                        hde[fva.loc[i]['Reaction_ID']] = [host, virus]
                        print('old bounds:', old_lb, old_ub)
                        print('Forward:', fva.loc[i]['Reaction_ID'], new_lower_bound, new_upper_bound)

                    elif new_lower_bound < 0 and new_upper_bound <= 0:
                        hde[fva.loc[i]['Reaction_ID']] = [host, virus]
                        print('old bounds:', old_lb, old_ub)
                        print('Reverse:', fva.loc[i]['Reaction_ID'], new_lower_bound, new_upper_bound)

                    elif new_lower_bound < 0 and new_upper_bound > 0:
                        hde[fva.loc[i]['Reaction_ID']] = [host, virus]
                        print('old bounds:', old_lb, old_ub)
                        print('Both directions:', fva.loc[i]['Reaction_ID'], new_lower_bound, new_upper_bound)

                    else:
                        print('old bounds:', old_lb, old_ub)
                        print('Unknown:', fva.loc[i]['Reaction_ID'], new_lower_bound, new_upper_bound)
                    
                print("------------------------------------------")        

    
    # remove BOF and VBOF if appear in targets
    if vbof_name in hde:
        hde.pop(vbof_name)
    if bof_name in hde:
        hde.pop(bof_name)
        

    print('Total HDE targets', len(hde))
    print() 
    
    hde_vbof = {}
    for key in hde.keys():
        hde_vbof[key] = (hde[key][1]/growth_rate_vbof)*100
    
    hde_vbof = {k: v for k, v in sorted(hde_vbof.items(), key=lambda item: item[1])}
    
    data = [[key, hde_vbof[key]] for key in hde_vbof.keys()]
    print('HDE Targets sorted:', data)
    print()
    print('HDE - top first target:', data[0])
    
    return data

#### Create VBOFs + introduce AA mutations + target prediction:

In [ ]:
count_iterations = 0

# store stoichiometric coefficients of all VBOF compounds of all input sequences
stoichiometric_coeffs = {} 

# store results of KO and HDE experiments of all input sequences
ko_results_all_sequences, hde_results_all_sequences = {}, {} 

# store all counts from all input sequences
amino_acids_counts = {}

for aa in amino_acids:
    amino_acids_counts[aa] = []

# for all input sequences
for ids, seq in all_sequences.items():
    
    nucl_aa = [] # store counts of nucleotides and amino acids in the current sequence
    
    print(f"Seq.Nr. {count_iterations}")
    print(f"Seq.ID. {ids}")
    
    count_dict = {}
    
    #********************************************
    # Nucleotides Investment
    #********************************************
    
    # count nucleotides in all sequences
    A, U, G, C = 0, 0, 0, 0

    # Get nucleotide count from .fasta file
    A = seq.count('A')
    U = seq.count('T')
    G = seq.count('G')
    C = seq.count('C')
    print("A,U,G,C: ",A,U,G,C)

    # compute moles of each nulceotide in a mole of virus particle
    AUtot = Cg*2*(A + U)
    GCtot = Cg*2*(G + C)
    print('AUtot: ', AUtot)
    print('GCtot: ', GCtot)

    # convert moles of nucleotides into grams of nucleotide per mole of virus
    GA = AUtot*135.13
    GU = AUtot*112.09
    GG = GCtot*151.13
    GC = GCtot*111.1

    # compute the mass of all genome components
    Gi = GA+GU+GG+GC
    print('Gi: ',Gi)    
    

    # Count Amino Acids in proteins
    for letter in amino_acids:
        num = 0
        for i in range(len(records)):
            if records[i].id == 'lcl|NC_045512.2_prot_YP_009724390.1_3': #spike
                num += records[i].seq.count(letter)*Cs
            elif records[i].id == 'lcl|NC_045512.2_prot_YP_009724392.1_5': #envelope
                num += records[i].seq.count(letter)*Ce
            elif records[i].id == 'lcl|NC_045512.2_prot_YP_009724393.1_6': #membrane
                num += records[i].seq.count(letter)*Cm
            elif records[i].id == 'lcl|NC_045512.2_prot_YP_009724397.2_11': #nucleocapsid
                num += records[i].seq.count(letter)*Cn
            else:
                num += records[i].seq.count(letter) 
        
        # for current amino acid
        count_dict[letter] = [num]

    # **********************************************************
    # Introduce Mutations in reference protein sequence 
    # **********************************************************   
    # if table with mutations is given
    if csv_table != " ":
        
        for mut in list(data_df[data_df['strain']==ids]['AA Mutations'])[0].split(","):

            if 'del' in mut or 'stop' in mut:

                to_delete = mut.strip().split(" ")[1][0]

                # consider different weights of structural proteins when deleting/inserting amino acid
                if 'Spike' in mut:
                    count_dict[to_delete][0] = count_dict[to_delete][0] - Cs
                elif "E " in mut: #e
                    count_dict[to_delete][0] = count_dict[to_delete][0] - Ce
                elif "M " in mut: #m
                    count_dict[to_delete][0] = count_dict[to_delete][0] - Cm
                elif "N " in mut: #n
                    count_dict[to_delete][0] = count_dict[to_delete][0] - Cn
                else:
                    count_dict[to_delete][0] = count_dict[to_delete][0] - 1   

            elif 'ins' in mut or 'dupl' in mut:

                to_delete = mut.strip().split(" ")[1][0]

                # consider differnt weights of structural proteins when deleting/inserting amino acid
                if 'Spike' in mut:
                    count_dict[to_delete][0] = count_dict[to_delete][0] + Cs
                elif "E " in mut: #e
                    count_dict[to_delete][0] = count_dict[to_delete][0] + Ce
                elif "M " in mut: #m
                    count_dict[to_delete][0] = count_dict[to_delete][0] + Cm
                elif "N " in mut: #n
                    count_dict[to_delete][0] = count_dict[to_delete][0] + Cn
                else:
                    count_dict[to_delete][0] = count_dict[to_delete][0] + 1

            else:

                to_delete = "".join(filter(str.isupper, mut.strip().split(" ")[1]))[0]
                to_add = "".join(filter(str.isupper, mut.strip().split(" ")[1]))[1]

                # consider differnt weights of structural proteins when deleting/inserting amino acid
                if 'Spike' in mut:
                    count_dict[to_delete][0] = count_dict[to_delete][0] - Cs
                    count_dict[to_add][0] = count_dict[to_add][0] + Cs
                elif "E " in mut: #e
                    count_dict[to_delete][0] = count_dict[to_delete][0] - Ce
                    count_dict[to_add][0] = count_dict[to_add][0] + Ce
                elif "M " in mut: #m
                    count_dict[to_delete][0] = count_dict[to_delete][0] - Cm
                    count_dict[to_add][0] = count_dict[to_add][0] + Cm

                elif "N " in mut: #n
                    count_dict[to_delete][0] = count_dict[to_delete][0] - Cn
                    count_dict[to_add][0] = count_dict[to_add][0] + Cn
                else:
                    count_dict[to_delete][0] = count_dict[to_delete][0] - 1
                    count_dict[to_add][0] = count_dict[to_add][0] + 1
    
    # store counts of amino acids
    for keys, values in count_dict.items():
        amino_acids_counts[keys].append(values[0])
    
    # ********************************************
    # Amino Acid Investment
    # ********************************************
    
    for letter, weight in zip(amino_acids, molecular_weight):
        count_dict[letter].append(count_dict[letter][0]*weight) 
    
    # Compute mass of all proteome components
    Gj = 0    
    for letter in amino_acids:
        Gj += count_dict[letter][1]

    print('Gj: ', Gj)
    
    #**********************
    # Total Viral Mass
    #**********************
    Mv = Gi+Gj
    print('Mv: ', Mv)


    SA = 1000*(AUtot/Mv) #stoichiometric coefficient of Adenine
    SG = 1000*(GCtot/Mv) #stoichiometric coefficient of Guanine
    print("SA: ", SA)
    print("SG: ", SG)

    for letter in amino_acids:
        count_dict[letter].append(1000*(count_dict[letter][0]/Mv)) #entry 4 = SjX

    #**********************
    # ATP Requirement
    #**********************

    new_count_dict = count_dict
    Xj = 0
    for letter in amino_acids:
        Xj += count_dict[letter][0]

    ATOT = (Xj * kATP) - kATP
    SATP = 1000*(ATOT/Mv)
    print('SATP: ', SATP)

    #*****************************
    # Pyrophosphate Liberation
    #*****************************
    PG = ((A + U + G + C)*kppi) - kppi
    PR = ((A + U + G + C)*kppi) - kppi

    PTOT = Cg*(PG+PR)

    SPPi = 1000*(PTOT/Mv)
    print('SPPi: ', SPPi)
    
    print('Amino acid counts: ', count_dict)
    
    #***************************************************************
    # VBOF Reconstruction + Incorporation in host-virus cell 
    #***************************************************************
    with model: 
        
        # add new VBOF
        reaction = Reaction('VBOF')
        reaction.name = 'Viral biomass objective function of coronavirus 2019-nCoV'
        reaction.lower_bound = 0 
        reaction.upper_bound = 1000 

        reaction.add_metabolites({
            #energy requirements
            model.metabolites.get_by_id('adp_c'): SATP,  
            model.metabolites.get_by_id('h_c'): SATP,  
            model.metabolites.get_by_id('h2o_c'): -SATP,  
            model.metabolites.get_by_id('pi_c'): SATP, 
            model.metabolites.get_by_id('ppi_c'): SPPi, 

            #nucleotides: left-hand terms 
            model.metabolites.get_by_id('atp_c'): -(SATP + SA), 
            model.metabolites.get_by_id('ctp_c'): -SG,
            model.metabolites.get_by_id('gtp_c'): -SG,
            model.metabolites.get_by_id('utp_c'): -SA,

            #amino acids: left-hand terms
            model.metabolites.get_by_id('ala__L_c'): -count_dict['A'][2],
            model.metabolites.get_by_id('arg__L_c'): -count_dict['R'][2], 
            model.metabolites.get_by_id('asn__L_c'): -count_dict['N'][2],
            model.metabolites.get_by_id('asp__L_c'): -count_dict['D'][2],
            model.metabolites.get_by_id('cys__L_c'): -count_dict['C'][2],
            model.metabolites.get_by_id('glu__L_c'): -count_dict['E'][2],
            model.metabolites.get_by_id('gln__L_c'): -count_dict['Q'][2],
            model.metabolites.get_by_id('gly_c'): -count_dict['G'][2],
            model.metabolites.get_by_id('his__L_c'): -count_dict['H'][2],
            model.metabolites.get_by_id('ile__L_c'): -count_dict['I'][2],
            model.metabolites.get_by_id('leu__L_c'): -count_dict['L'][2],
            model.metabolites.get_by_id('lys__L_c'): -count_dict['K'][2],
            model.metabolites.get_by_id('met__L_c'): -count_dict['M'][2],
            model.metabolites.get_by_id('phe__L_c'): -count_dict['F'][2],
            model.metabolites.get_by_id('pro__L_c'): -count_dict['P'][2],
            model.metabolites.get_by_id('ser__L_c'): -count_dict['S'][2],
            model.metabolites.get_by_id('thr__L_c'): -count_dict['T'][2],
            model.metabolites.get_by_id('trp__L_c'): -count_dict['W'][2],
            model.metabolites.get_by_id('tyr__L_c'): -count_dict['Y'][2],
            model.metabolites.get_by_id('val__L_c'): -count_dict['V'][2], 
            
            # add lipids
            model.metabolites.pchol_hs_c: 0.038400, 
            model.metabolites.pe_hs_c: 0.014566, 
            model.metabolites.pail_hs_c: 0.006621, 
            model.metabolites.ps_hs_c: 0.001986, 
            model.metabolites.chsterol_c: 0.000012, 
            model.metabolites.sphmyln_hs_c: 0.001986 

        })

        model.add_reactions([reaction])

        s_matrix = cobra.util.array.create_stoichiometric_matrix(model=model, array_type='DataFrame')
        # store stoichiometric coeffs as absolute values
        stoichiometric_coeffs[ids] = list(map(abs,list(s_matrix['VBOF'].loc[s_matrix['VBOF'] != 0])))
        # store compounds participating in VBOF
        vbof_compounds = list(s_matrix['VBOF'].loc[s_matrix['VBOF'] != 0].index)

        print("VBOF compounds: ",vbof_compounds)

        #***************************************
        # Targets Prediction Experiments
        #***************************************

        # (1) Single reaction knock-out experiments
        print("\n * Single Reaction Knock-outs * \n")
        
        # optimize host
        model.objective = 'biomass_bec'
        growth_rate_bec = model.optimize().objective_value
        react_ko_mac = single_reaction_deletion(model)
        react_ko_mac = react_ko_mac.rename(columns={'growth': 'growth_host', 'status': 'status_host'})
        # change indices from numbers to reaction IDs
        new_indices = []
        for i in react_ko_mac["ids"]:
            new_indices.append(list(i)[0])
        react_ko_mac.set_index([pd.Index(new_indices)], inplace=True)

        # optimize virus
        model.objective = 'VBOF'
        growth_rate_vbof = model.optimize().objective_value
        react_ko_vbof = single_reaction_deletion(model)
        react_ko_vbof = react_ko_vbof.rename(columns={'growth': 'growth_virus', 'status': 'status_virus'})
        # change indices from numbers to reaction IDs
        new_indices2 = []
        for i in react_ko_vbof["ids"]:
            new_indices2.append(list(i)[0])
        react_ko_vbof.set_index([pd.Index(new_indices2)], inplace=True)

        # merge results from BOF and VBOF knock-out
        growth_virus = react_ko_vbof["growth_virus"]
        merged = react_ko_mac.join(growth_virus)

        # for each reaction: calculate what percentage of host growth remains after single reaction knock-out
        percent_bec = []
        for i in merged.index:
            percent_bec.append(merged.loc[i]['growth_host']/growth_rate_bec*100)
        merged.insert(loc = 0, column='%growth_host', value=percent_bec)

        percent_vbof = []
        for i in merged.index:
            percent_vbof.append(merged.loc[i]['growth_virus']/growth_rate_vbof*100)     
        merged.insert(loc = 0, column='%growth_virus', value=percent_vbof)

        # keep only interesting -- > host 99% and virus 0%
        print('KO Targets: ', list(merged.loc[(merged['%growth_host'] > 99) & (merged['%growth_virus'] == 0.0)].index))
        ko_results_all_sequences[ids] = list(merged.loc[(merged['%growth_host'] > 99) & (merged['%growth_virus'] == 0.0)].index)


        #######################################
        # Host-derived enforcement
        #######################################
        hde_results_all_sequences[ids] = host_derived_enforcement(model,'VBOF','biomass_bec',growth_rate_vbof,growth_rate_bec)

    print('\n')
    count_iterations += 1

#### Collect all predicted targets + store results + visualize:

In [ ]:
all_targets = []

for keys,values in hde_results_all_sequences.items():
    for lst in values:
        all_targets.append(lst[0])

# store targest that were predicted for all the input sequences
all_unique_targets = list(set(all_targets))
print(len(all_unique_targets))
print(all_unique_targets)

In [ ]:
# create dataframes with HDE results for all input sequences
if csv_table != " ":
    
    merged = pd.DataFrame(columns=all_unique_targets, index=list(hde_results_all_sequences.keys()))

    c=1
    gisaid_ids = []

    for seq_id in all_sequences.keys():

        gisaid_ids.append(data_df.loc[data_df['strain'] == seq_id]['gisaid_epi_isl'].item())

        seq = hde_results_all_sequences[seq_id]
        seq_targets = [l[0] for l in seq]
        seq_remaining_virus = [l[1] for l in seq]

        seq_df = pd.DataFrame(columns=['Targets', '% remaining VBOF'])
        seq_df['Targets'] = seq_targets
        seq_df['% remaining VBOF'] = seq_remaining_virus
        # sort by target ID
        seq_df.sort_values(by=['Targets'], inplace=True)
        # store %remaining VBOF in merged df
        for r in all_unique_targets:
            if r in list(seq_df['Targets']):
                merged[r][seq_id] = list(seq_df.loc[seq_df['Targets']==r]['% remaining VBOF'])[0]
            else:
                merged[r][seq_id] = np.nan

        # sort by value
        seq_df.sort_values(by=['% remaining VBOF'], inplace=True)

        # store as csv file
        seq_df.to_csv(path+variant_name+'_seq_'+str(c)+'.csv')

        c+=1

    merged.index = gisaid_ids

    # get mean of targets
    mean_of_targets = []
    for (columnName, columnData) in merged.iteritems():

        mean_of_targets.append(columnData.values.mean())
    print('mean_of_targets:', mean_of_targets)

    merged = merged.T
    merged['Mean'] = mean_of_targets

    merged.to_csv(path+variant_name+'_all_merged_HDE_results_'+variant_name+'.csv')

    # remove target names form index to be able to acess them below
    merged = merged.reset_index()
    merged.rename({'index': 'Target'}, axis=1, inplace=True)
    print(merged)
    
    # NaN in mean column means that mean could be computed because 
    # the reaction for some sequences was not observed as a target
    # thus it is not robust.
    # --> look only those predicted for all input sequences as targets

    robust_targets =merged[merged['Mean'].notna()]
    robust_targets.to_csv(path+variant_name+'_robust_targets.csv')
    print(robust_targets)
    
    #######################
    # Visualize
    #######################
    plt.figure(figsize=(12,4))

    robust_targets = robust_targets.sort_values(by='Mean',ascending=False)

    plt.bar(robust_targets['Target'], height = robust_targets['Mean'], width = 0.1, color="#41b6c4") 
    plt.plot(robust_targets['Target'], robust_targets['Mean'], 'o', color="#253494")


    plt.xticks(rotation=90, fontsize=12)
    plt.xlabel("")

    plt.yticks(fontsize=12)
    plt.ylabel("% remaining virus growth", fontsize=14)

    plt.xlim([-0.5,len(robust_targets['Target'])])

    plt.ylim(31,47)


    plt.tight_layout()
    plt.savefig(path+variant_name+'_all_HDE_targets_mean_after_results.pdf')

    plt.show()

# if only one sequence in input 
else:
    
    targets = [j[0] for i in list(hde_results_all_sequences.values()) for j in i]
    remaining_virus = [j[1] for i in list(hde_results_all_sequences.values()) for j in i]
    
    merged = pd.DataFrame(remaining_virus,targets, columns=['%Remaining_virus'])
    merged.to_csv(path+variant_name+'_HDE_targets.csv')
    print(merged)
    
    #######################
    # Visualize
    #######################
    plt.figure(figsize=(12,4))

    sorted_targets = merged.sort_values(by='%Remaining_virus',ascending=False)

    plt.bar(list(sorted_targets.index), height = sorted_targets['%Remaining_virus'], width = 0.1, color="#41b6c4") 
    plt.plot(list(sorted_targets.index), sorted_targets['%Remaining_virus'], 'o', color="#253494")


    plt.xticks(rotation=90, fontsize=12)
    plt.xlabel("")

    plt.yticks(fontsize=12)
    plt.ylabel("% remaining virus growth", fontsize=14)

    plt.tight_layout()
    plt.savefig(path+variant_name+'_all_HDE_targets_mean_after_results.pdf')

    plt.show()


In [ ]:
vbof_compounds = ['ala__L_c', 'adp_c', 'arg__L_c', 'asn__L_c', 'asp__L_c', 'atp_c', 'ctp_c', 'cys__L_c', 'gln__L_c', 'glu__L_c', 'gly_c', 'h2o_c', 'h_c', 'gtp_c', 'his__L_c', 'ile__L_c', 'lys__L_c', 'leu__L_c', 'met__L_c', 'pi_c', 'ppi_c', 'pro__L_c', 'phe__L_c', 'ser__L_c', 'thr__L_c', 'tyr__L_c', 'trp__L_c', 'utp_c', 'val__L_c']

if csv_table!= " ":
    vbofs_info = pd.DataFrame(stoichiometric_coeffs).T
    vbofs_info.columns = vbof_compounds
    vbofs_info.index = data_df['gisaid_epi_isl']
    vbofs_info.to_csv(path+variant_name+'_all_20_VBOFs_stoich_coeffs_info.csv')
    print(vbofs_info)
else:
    vbofs_info = pd.DataFrame(stoichiometric_coeffs).T
    vbofs_info.columns = vbof_compounds
    
    vbofs_info.to_csv(path+variant_name+'_all_VBOFs_stoich_coeffs_info.csv')

    print(vbofs_info)

In [ ]:
if csv_table!=" ":
    info = pd.DataFrame(columns=["GISAID Accession ID", "Adenine (A)", "Thymine (T)", "Guanine (G)", "Cytosine (C)"])

    info['GISAID Accession ID'] = nucleotide_counts_df['GISAID Accession ID']
    info["Adenine (A)"] = nucleotide_counts_df['A']
    info["Thymine (T)"] = nucleotide_counts_df['T']
    info["Guanine (G)"] = nucleotide_counts_df['G']
    info["Cytosine (C)"] = nucleotide_counts_df['C']
    
else:
    info = pd.DataFrame(columns=["Adenine (A)", "Thymine (T)", "Guanine (G)", "Cytosine (C)"])

    info.loc[0] = [list(all_sequences.values())[0].count("A"),list(all_sequences.values())[0].count("T"),
                       list(all_sequences.values())[0].count("G"),list(all_sequences.values())[0].count("C")]
    
for aa in amino_acids:
    info[aa] = amino_acids_counts[aa]
info.to_csv(path+variant_name+"_info_sequences.csv")
info